In [1]:
from transformers import pipeline
import pandas as pd
from datasets import Dataset, load_metric
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer
import numpy as np
import pandas as pd

from huggingface_hub import notebook_login

In [3]:
modello = pipeline(model="DayOne")  #Carica il modello da huggingface
test_pd = pd.read_csv("data/processed_dataset/test.csv")

Selezione prima 512 token del campo Processed_text

In [4]:
from transformers import AutoTokenizer

# Load the tokenizer
checkpoint = 'nlpaueb/legal-bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Assuming 'test_pd' is your DataFrame with a 'Processed_text' column
processed_text_column = test_pd['Processed_text']

# Tokenize and remove [CLS] and [SEP] tokens for each element in the column
tokenized_texts = [tokenizer(text, truncation=True, padding=True, max_length=512) for text in processed_text_column]

# Remove [CLS] and [SEP] tokens from the end of each tokenized text
for tokenized_text in tokenized_texts:
    if tokenized_text['input_ids'][-1] == tokenizer.sep_token_id:
        tokenized_text['input_ids'] = tokenized_text['input_ids'][:-1]
    if tokenized_text['input_ids'][0] == tokenizer.cls_token_id:
        tokenized_text['input_ids'] = tokenized_text['input_ids'][1:]

# Convert tokenized texts back to normal texts
decoded_texts = [tokenizer.decode(tokenized_text['input_ids']) for tokenized_text in tokenized_texts]

# Replace the original 'Processed_text' column with the decoded texts
test_pd['truncated_text'] = decoded_texts

In [5]:
output = modello.predict(test_pd["truncated_text"].tolist())

Salva il risultato in un file .csv

In [6]:
etichette = [int(dizionario['label'].split('_')[1]) for dizionario in output]

# Crea un DataFrame con la colonna 'etichette'
df = pd.DataFrame({'label': etichette})
df['label'] = df['label']+1

df.to_csv("data/result.csv", index=False)